# Towards an Applicative for macro

Some time ago with the release of the Haskell's [Haxl](https://hackage.haskell.org/package/haxl) library the same authors realized that the `do` notation in Haskell may need some improvements. The `do` notation can be used to "flatten" deep nestings of `>>=` applications in Haskell. For example instead of writting something like this:

```haskell
findVal :: String -> Maybe Int
findVal key = ... -- some function definition

sum :: Maybe Int
sum = (findVal "key1") >>= (\val1 -> (findVal "key2") >>= (\val2 -> return (val1 + val2))))
```
you can write:

```haskell
sum :: Maybe Int
sum = do val1 <- findVal "key1"
         val2 <- findVal "key2"
         return (val1 + val2)
```

This makes it easier to read the expression and to understand what's going on. Haskell's compiler simply desugars this notation to `>>=` applications. The `do` notation is specially readable because you can interpret it as follows: At the left side of a `<-` you will find the "extracted" value of the monadic value at the right side.

Now as you may have noticed in this example we are not using the whole power of the `Monad` typeclass. We are just calling `>>=` because we want to have the values inside the monadic values in the same place so we can express our desired computation: `(val1 + val2)`. But `Monad` is more powerful than that. For example we can express something like this:

```haskell
do val1 <- findVal "key1"
   val2 <- findVal ("key" ++ show val1)
   return val2
```

In this case the second monadic value depends on the result of the first monadic value. Unlike the previous example this can only be achieved with something as powerful as `Monad`.

When programming you want to use the least powerful abstraction and in functional programming this translates into using the most general typeclass so your function can be reused in many situations. In this case the least powerful abstraction that allows us to join values in different contexts into one is `Applicative`. So we could rewrite the first example like this:

```haskell
add x y = x + y

sum = add <$> (findVal "key1") <*> (findVal "key2")
```

If you are thinking this is less readable than the `do` version you are on to something. As it turns out for the `Maybe` monad there is not much difference between choosing one style or the other because both alternatives have a similar performance. But there are other monads in which it is preferrable to use `Applicative` when possible. The first one that comes to my mind is precisely Haxl's `Fetch` monad. With `Fetch` applicative independent computations can be done concurrently and even more if they access the same data source then the query can be batched.

For instance in the following situation:

```haskell
fetchData :: String -> Fetch Data
fetchData key = ... -- build fetch value

join x y = (x,y)

dataTuple :: (Data,Data)
dataTuple = join <$> fetchData "key1" <*> fetchData "key2"
```

The same final result can be expressed with `do`:

```haskell
do data1 <- fetchData "key1"
   data2 <- fetchData "key2"
   return (data1,data2)
```

But there is a catch: given that in this situation the `do` expression is desugared into `>>=` applications then the fetches are done sequentially. This is because the first monadic value must be computed before computing the one in the second line. Thus, by trying to use a more intuitive and readable notation we are incurring in a performance cost. 

But the problem is not just readability. When you are making changes to a codebase you must be aware that the change may allow you to use `Applicative` instead of monad. This may happen if, for example you are changing a line of a `do` expression.

Wouldn't it be cool that `do` expressions would use `Monad` just when they are strictly necessary and `Applicative` were possible? This would allow us to always use the `do` notation without having to worry if we are using the correct typeclass. This is precisely the topic of Haxl's follow-up [paper](http://research.microsoft.com/en-us/um/people/simonpj/papers/list-comp/applicativedo.pdf). There is a [proposal](https://ghc.haskell.org/trac/ghc/wiki/ApplicativeDo) in the Haskell community to desugar using `Applicative` when possible.

Now, let's see in Scala how useful this could be. Remember that Scala's `for` comprehensions are similar to Haskell's `do` expressions.

Let's start with our very own definition of a `Validation` which will be similar to Scalaz's `ValidationNel` or Cat's `ValidatedNel`:

In [1]:
sealed trait Validation[+A] {
    def flatMap[B](f: A => Validation[B]): Validation[B] = this match {
        case Success(value) => f(value)
        case Failure(error) => Failure(error)
    }
    def map[B](f: A => B): Validation[B] = flatMap(a => Success(f(a)))
}
case class Success[A](value: A) extends Validation[A]
case class Failure(errors: List[String]) extends Validation[Nothing]

defined trait Validation
defined class Success
defined class Failure

Now let's define the `Monad` and `Applicative` typeclasses and then describe the instances for `Validation`:

In [2]:
trait Monad[F[_]] {
    def unit[A](a: A): F[A]
    def flatMap[A,B](fa: F[A])(f: A => F[B]): F[B]
}

trait Applicative[F[_]] {
    def pure[A](a: A): F[A]
    def map2[A,B,C](fa: F[A], fb: F[B])(f: (A,B) => C): F[C]
}

implicit object ValidationMonad extends Monad[Validation] {
    def unit[A](a: A) = Success(a)
    def flatMap[A,B](va: Validation[A])(f: A => Validation[B]): Validation[B] =
        va.flatMap(f)
}

implicit object ValidationApplicative extends Applicative[Validation] {
    def pure[A](a: A) = Success(a)
    def map2[A,B,C](va: Validation[A], vb: Validation[B])(f: (A,B) => C): Validation[C] = {
        (va, vb) match {
            case (Success(a) , Success(b) ) => Success(f(a,b))
            case (Failure(ea), Failure(eb)) => Failure(ea ++ eb)
            case (Failure(ea), _          ) => Failure(ea)
            case (_          , Failure(eb)) => Failure(eb)            
        }
    }
}

defined trait Monad
defined trait Applicative
defined object ValidationMonad
defined object ValidationApplicative

As you may have noticed my definition of `Applicative` is a little bit different from the usual formulation which defines a function `ap` (with type `F[A => B] => F[A] => F[B]`). As it turns out both formulations are equivalent: you can convince yourself by implementing one in terms of the other.

Another thing to notice is that when `flatMap` fails it will fail with the error of the first `Validation` value. In contrast the `Applicative` instance says that if both of the values are failed then we can accumulate them.

Let's see an example. First with `flatMap`:

In [3]:
val v1: Validation[Int] = Failure(List("error1"))
val v2: Validation[Int] = Failure(List("error2"))

val withFlatMap = for {
    x <- v1
    y <- v2
} yield x + y

v1: Validation[Int] = Failure(List(error1))
v2: Validation[Int] = Failure(List(error2))
withFlatMap: Validation[Int] = Failure(List(error1))

In this case the error in the result value is the one in the first validation. But given that the value `v2` doesn't depend on the value `x` couldn't we report also the failure of `v2`? Let's see  what `Applicative` can do:

In [4]:
val withApplicative = ValidationApplicative.map2(v1,v2)(_ + _)

withApplicative: Validation[Int] = Failure(List(error1, error2))

Now, let's imagine we have a web form with a bunch of fields, each one of which has to be validated. But when the form is submitted and contains errors we don't want to bother the user by just reporting the first error. We would like to report the majority of independent errors:

```scala
for {
    okFirstName <- validateFirstNameField
    okLastName  <- validateLastNameField
    okFullName  <- validateFullName(okFirstName, okLastName)
    okAge       <- validateAge
} yield NewUserData(okFirstName, okLastName, okFullName, okAge)
```

If we used a for comprehension we would be making a mistake because if there is one error only that one is going to be returned. We can use Applicative and some syntactic sugar like the one in Scalaz to get something like this:

```scala
for {
    (okFirstName, okLastName, okAge) <- (validateFirstNameField |@| validateLastNameField |@| validateAge).tupled
    okFullName                       <- validateFullName(okFirstName, okLastName)
} yield NewUserData(okFirstName, okLastName, okFullName, okAge)
```

This works but it may be less readable. More importantly this is more brittle to future changes and coupled to the current computation structure. You can imagine what may happen with more fields and more complex dependencies between those fields.

## Towards an Applicative macro

I would be very useful if this could be done automatically by the compiler. In Scala the for comprehensions syntax is just another phase of the compiler. So when a macro inspects this code it will be already desugared into a nested sequence of `flatMap`s and `map`s. Let's see if we can build a macro that replaces `flatMap`s and `map`s by `map2`s when possible. 

We are going to start with a very simple example:

In [5]:
val v1: Validation[Int] = Failure(List("error1"))
val v2: Validation[Int] = Failure(List("error2"))

for {
    x <- v1
    y <- v2
} yield x + y

v1: Validation[Int] = Failure(List(error1))
v2: Validation[Int] = Failure(List(error2))
res4_2: Validation[Int] = Failure(List(error1))

Let's inspect the tree generated by this for comprehension:

In [6]:
import scala.reflect.runtime.universe._

val tree = reify {
    for {
        x <- v1
        y <- v2
    } yield x + y
}.tree

showRaw(tree)

import scala.reflect.runtime.universe._
tree: reflect.runtime.package.universe.Tree = cmd5.$ref$cmd4.v1.flatMap(((x) => cmd5.$ref$cmd4.v2.map(((y) => x.$plus(y)))))
res5_2: String = """
Apply(Select(Select(Select(Ident(TermName("cmd5")), TermName("$ref$cmd4")), TermName("v1")), TermName("flatMap")), List(Function(List(ValDef(Modifiers(PARAM), TermName("x"), TypeTree(), EmptyTree)), Apply(Select(Select(Select(Ident(TermName("cmd5")), TermName("$ref$cmd4")), TermName("v2")), TermName("map")), List(Function(List(ValDef(Modifiers(PARAM), TermName("y"), TypeTree(), EmptyTree)), Apply(Select(Ident(TermName("x")), TermName("$plus")), List(Ident(TermName("y"))))))))))
"""

The part that interests us is the `flatMap` application:

In [7]:
val Apply(Select(firstMonadicValue, TermName("flatMap")), List(functionDef)) = tree

firstMonadicValue: Tree = cmd5.$ref$cmd4.v1
functionDef: Tree = ((x) => cmd5.$ref$cmd4.v2.map(((y) => x.$plus(y))))

Now we must separate the function into it's argument and it's body:

In [8]:
showRaw(functionDef)
val Function(List(firstArgumentTerm), functionBody) = functionDef 
//               ^ only works for functions of arity one, which works for map and flatMap

res7_0: String = """
Function(List(ValDef(Modifiers(PARAM), TermName("x"), TypeTree(), EmptyTree)), Apply(Select(Select(Select(Ident(TermName("cmd5")), TermName("$ref$cmd4")), TermName("v2")), TermName("map")), List(Function(List(ValDef(Modifiers(PARAM), TermName("y"), TypeTree(), EmptyTree)), Apply(Select(Ident(TermName("x")), TermName("$plus")), List(Ident(TermName("y"))))))))
"""
firstArgumentTerm: ValDef = val x = _
functionBody: Tree = cmd5.$ref$cmd4.v2.map(((y) => x.$plus(y)))

Now, if the `functionBody` consists in calling `map` or `flatMap` over some expression, then we must identify if that expression uses `termArgument`. If that's the case then we can't do anything, one expression depends on the other and `flatMap` is the right function. But if not, then that's an opportunity to use `Applicative`'s `ap` instead of `Monad`'s `flatMap`/`bind`. Let's first define a function `usesTerm` that will indicate if a term is used in a expression:

In [9]:
def usesTerm(term: ValDef, exp: Tree): Boolean = {
    val ValDef(_,termName,_,_) = term
    //println(s"**${showRaw(term)}")
    exp.find{ x => 
             //println(s"--${showRaw(x)}")
             x match {
                case Ident(_termName) if termName == _termName => 
                    true
                case _ => 
                    false
             }
    }.isDefined
}

defined function usesTerm

Now let's separate the `functionBody` into two parts: the second monadic value and the next function definition

In [10]:
showRaw(functionBody)
val Apply(Select(secondMonadicValue, TermName("map")), List(secondFunctionDef)) = functionBody
showRaw(firstArgumentTerm)

res9_0: String = """
Apply(Select(Select(Select(Ident(TermName("cmd5")), TermName("$ref$cmd4")), TermName("v2")), TermName("map")), List(Function(List(ValDef(Modifiers(PARAM), TermName("y"), TypeTree(), EmptyTree)), Apply(Select(Ident(TermName("x")), TermName("$plus")), List(Ident(TermName("y")))))))
"""
secondMonadicValue: Tree = cmd5.$ref$cmd4.v2
secondFunctionDef: Tree = ((y) => x.$plus(y))
res9_2: String = """
ValDef(Modifiers(PARAM), TermName("x"), TypeTree(), EmptyTree)
"""

And we are interested in answering the question: is the "extracted" value for the first monad being used when defining the second monadic value?

In [11]:
usesTerm(firstArgumentTerm, secondMonadicValue)

res10: Boolean = false

Ok so as we can see for computing the second term we don't need the function argument. So we want to transform this `flatMap`->`map` call into an `Applicative`'s `map2` call. For this we will need to extract the innermost expression of the for comprehension, that is the `x+y` expression. After that we will have to build the `map2` call passing the appropiate arguments. So first let's extract that expression from `nextFunctionDef`: 

In [12]:
showRaw(secondFunctionDef)
val Function(List(secondArgumentTerm), innerExpr) = secondFunctionDef

res11_0: String = """
Function(List(ValDef(Modifiers(PARAM), TermName("y"), TypeTree(), EmptyTree)), Apply(Select(Ident(TermName("x")), TermName("$plus")), List(Ident(TermName("y")))))
"""
secondArgumentTerm: ValDef = val y = _
innerExpr: Tree = x.$plus(y)

This are all the ingredients we need:

In [13]:
showRaw(firstMonadicValue)
showRaw(firstArgumentTerm)
showRaw(secondMonadicValue)
showRaw(secondArgumentTerm)
showRaw(innerExpr)

res12_0: String = """
Select(Select(Ident(TermName("cmd5")), TermName("$ref$cmd4")), TermName("v1"))
"""
res12_1: String = """
ValDef(Modifiers(PARAM), TermName("x"), TypeTree(), EmptyTree)
"""
res12_2: String = """
Select(Select(Ident(TermName("cmd5")), TermName("$ref$cmd4")), TermName("v2"))
"""
res12_3: String = """
ValDef(Modifiers(PARAM), TermName("y"), TypeTree(), EmptyTree)
"""
res12_4: String = """
Apply(Select(Ident(TermName("x")), TermName("$plus")), List(Ident(TermName("y"))))
"""

In [15]:
val result = q"""
ValidationApplicative.map2(
    $firstMonadicValue,
    $secondMonadicValue,
    ${Function(List(firstArgumentTerm, secondArgumentTerm), innerExpr)}
)
"""
showRaw(result)

result: Tree = ValidationApplicative.map2(cmd5.$ref$cmd4.v1, cmd5.$ref$cmd4.v2, ((x, y) => x.$plus(y)))
res14_1: String = """
Apply(Select(Ident(TermName("ValidationApplicative")), TermName("map2")), List(Select(Select(Ident(TermName("cmd5")), TermName("$ref$cmd4")), TermName("v1")), Select(Select(Ident(TermName("cmd5")), TermName("$ref$cmd4")), TermName("v2")), Function(List(ValDef(Modifiers(PARAM), TermName("x"), TypeTree(), EmptyTree), ValDef(Modifiers(PARAM), TermName("y"), TypeTree(), EmptyTree)), Apply(Select(Ident(TermName("x")), TermName("$plus")), List(Ident(TermName("y")))))))
"""

Unfortunately I haven't found a way to compile and execute expressions from the REPL.